In [8]:
pip install google-analytics-data
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/166.9 kB ? eta -:--:--
   --------- ------------------------------ 41.0/166.9 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 166.9/166.9 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.4/4.1 MB 13.5 MB/s eta 0:00:01
   ---------- ----------------------------- 1.1/4.1 MB 13.5 MB/s eta 0:00:01
   -------------------- ------------------- 2.1/4.1 MB 14.7 MB/s eta 0:00:01
   ---------------------------- ----------- 3.0/4.1 MB 17.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.8/4.1 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------  4.1/4.1 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------- 4.1/4.1 MB 15.3 MB/s eta 0:00:00
   ---------

Session Level data - Views and Page path

In [2]:
import json
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest
import pandas as pd
import config

# Reload the config module (useful if you've made changes to config.py and want to reload it in Jupyter Notebook)
#import importlib
#importlib.reload(config)

# Path to your service account key file
KEY_FILE_LOCATION = config.KEY_FILE_LOCATION

# Google Analytics property ID
PROPERTY_ID = config.PROPERTY_ID

# Define the credentials and initialize the Analytics Data API client
credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION)
client = BetaAnalyticsDataClient(credentials=credentials)

# Define the date range
date_ranges = [DateRange(start_date="2024-01-01", end_date="2024-01-01")]

# Define dimensions and metrics
dimensions = [
    Dimension(name="date"),
    Dimension(name="sessionDefaultChannelGroup"),
    Dimension(name="sessionSourceMedium"),
    Dimension(name="unifiedPagePathScreen"),
    Dimension(name="sessionPrimaryChannelGroup"),
    #Dimension(name="eventName"),
    Dimension(name="deviceCategory"),
    Dimension(name="country"),
    Dimension(name="city"),
 

]

metrics = [
    Metric(name="sessions"),
    Metric(name="screenPageViews"),
    Metric(name="engagedSessions"),
    Metric(name="userEngagementDuration"),
    Metric(name="eventCount"),
    #Metric(name="activeUsers")
    #Metric(name="")
]

# Create the API request
request = RunReportRequest(
    property=f"properties/{PROPERTY_ID}",
    date_ranges=date_ranges,
    dimensions=dimensions,
    metrics=metrics
)

# Make the request and handle potential errors
try:
    response = client.run_report(request)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Process the response and convert it to a DataFrame
def process_response(response):
    data = []
    for row in response.rows:
        row_data = {}
        for i, dimension_value in enumerate(row.dimension_values):
            row_data[dimensions[i].name] = dimension_value.value
        for i, metric_value in enumerate(row.metric_values):
            row_data[metrics[i].name] = metric_value.value
        data.append(row_data)
    return pd.DataFrame(data)

# Print the DataFrame
df1 = process_response(response)
df1.sort_values(by="date", inplace=True)
df1

# Assuming df is your DataFrame
num_rows = df1.shape[0]  # Using shape attribute
# Or
num_rows = len(df1)  # Using len() function

print("Number of rows in DataFrame:", num_rows)

df1

Number of rows in DataFrame: 776


,date,sessionDefaultChannelGroup,sessionSourceMedium,unifiedPagePathScreen,sessionPrimaryChannelGroup,deviceCategory,country,city,sessions,screenPageViews,engagedSessions,userEngagementDuration,eventCount
0,20240101,Direct,(direct) / (none),/our-latest-thinking,Direct,desktop,(not set),(not set),30,30,0,0,118
511,20240101,Organic Search,google / organic,/our-latest-thinking/publications/2021/08/data...,Organic Search,mobile,Hong Kong,(not set),1,1,1,47,7
512,20240101,Organic Search,google / organic,/our-latest-thinking/publications/2021/11/risk...,Organic Search,desktop,United States,(not set),1,1,1,22,7
513,20240101,Organic Search,google / organic,/our-latest-thinking/publications/2023/08/a-gu...,Organic Search,desktop,Canada,Vancouver,1,1,1,35,9
514,20240101,Organic Search,google / organic,/our-latest-thinking/publications/2023/08/fore...,Organic Search,desktop,Canada,Calgary,1,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,20240101,Organic Search,google / organic,/about-us/news-and-media/2023/12/torys-announc...,Organic Search,mobile,Canada,Montreal,1,1,1,33,4
263,20240101,Organic Search,google / organic,/about-us/news-and-media/2023/12/torys-announc...,Organic Search,mobile,Canada,Nicolet,1,3,1,2,8
264,20240101,Organic Search,google / organic,/about-us/news-and-media/2023/12/torys-announc...,Organic Search,mobile,Canada,Port Colborne,1,2,1,19,7
242,20240101,Organic Search,google / organic,/,Organic Search,mobile,Canada,Vancouver,1,1,1,3,3


In [18]:

# Export DataFrame to an Excel file
df1.to_excel('user_pagep_views.xlsx', index=False)

print("DataFrame has been exported to 'output.xlsx'")


DataFrame has been exported to 'output.xlsx'


Session Level data - without page path data and views

In [24]:
import json
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest
import pandas as pd
import config

# Reload the config module (useful if you've made changes to config.py and want to reload it in Jupyter Notebook)
#import importlib
#importlib.reload(config)

# Path to your service account key file
KEY_FILE_LOCATION = config.KEY_FILE_LOCATION

# Google Analytics property ID
PROPERTY_ID = config.PROPERTY_ID

# Define the credentials and initialize the Analytics Data API client
credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION)
client = BetaAnalyticsDataClient(credentials=credentials)

# Define the date range
date_ranges = [DateRange(start_date="2024-01-01", end_date="2024-01-01")]

# Define dimensions and metrics
dimensions = [
    Dimension(name="date"),
    Dimension(name="sessionDefaultChannelGroup"),
    Dimension(name="sessionSourceMedium"),
    #Dimension(name="unifiedPagePathScreen"),
    Dimension(name="sessionPrimaryChannelGroup"),
    #Dimension(name="deviceCategory"),
    #Dimension(name="country"),
    #Dimension(name="city"),
    Dimension(name="deviceCategory"),
]

metrics = [
    Metric(name="sessions"),
    Metric(name="screenPageViews"),
    Metric(name="engagedSessions"),
    Metric(name="userEngagementDuration"),
    Metric(name="eventCount"),
]

# Create the API request
request = RunReportRequest(
    property=f"properties/{PROPERTY_ID}",
    date_ranges=date_ranges,
    dimensions=dimensions,
    metrics=metrics
)

# Make the request and handle potential errors
try:
    response = client.run_report(request)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Process the response and convert it to a DataFrame
def process_response(response):
    data = []
    for row in response.rows:
        row_data = {}
        for i, dimension_value in enumerate(row.dimension_values):
            row_data[dimensions[i].name] = dimension_value.value
        for i, metric_value in enumerate(row.metric_values):
            row_data[metrics[i].name] = float(metric_value.value)
        data.append(row_data)
    return pd.DataFrame(data)

# Print the DataFrame
df2 = process_response(response)
df2.sort_values(by="date", inplace=True)

# Group by session-related dimensions and aggregate metrics to avoid duplication
session_related_dimensions = ["date", "sessionDefaultChannelGroup", "sessionSourceMedium", "sessionPrimaryChannelGroup"]
df_grouped = df2.groupby(session_related_dimensions).agg({
    "sessions": "sum",
    "screenPageViews": "sum",
    "engagedSessions": "sum",
    "userEngagementDuration": "sum"
}).reset_index()

# Print the number of rows and the grouped DataFrame
num_rows = df_grouped.shape[0]  # Using shape attribute
print("Number of rows in grouped DataFrame:", num_rows)

df2


Number of rows in grouped DataFrame: 26


,date,sessionDefaultChannelGroup,sessionSourceMedium,sessionPrimaryChannelGroup,deviceCategory,sessions,screenPageViews,engagedSessions,userEngagementDuration,eventCount
0,20240101,Organic Search,google / organic,Organic Search,mobile,196.0,366.0,134.0,9211.0,1776.0
21,20240101,Direct,(direct) / (none),Direct,tablet,1.0,1.0,0.0,0.0,4.0
22,20240101,Email,ActiveCampaign / email,Email,desktop,1.0,1.0,0.0,0.0,4.0
23,20240101,Organic Search,baidu / organic,Organic Search,mobile,1.0,1.0,0.0,0.0,4.0
24,20240101,Organic Search,bing / organic,Organic Search,mobile,1.0,1.0,1.0,20.0,6.0
25,20240101,Organic Search,ca.search.yahoo.com / referral,Organic Search,desktop,1.0,1.0,1.0,11.0,4.0
26,20240101,Organic Search,duckduckgo / organic,Organic Search,mobile,1.0,1.0,1.0,191.0,8.0
27,20240101,Organic Search,ecosia.org / organic,Organic Search,mobile,1.0,3.0,1.0,297.0,19.0
20,20240101,Organic Social,LinkedIn / social,Organic Social,desktop,2.0,2.0,0.0,0.0,9.0
28,20240101,Organic Search,yahoo / organic,Organic Search,desktop,1.0,2.0,1.0,55.0,9.0


In [ ]:
# Export DataFrame to an Excel file
df2.to_excel('user_sessions.xlsx', index=False)

print("DataFrame has been exported")